## Preparation + build model

In [1]:
import numpy as np
import pandas as pd
Train_df = pd.read_csv('./data/train_newfeature.csv')
Valid_df = pd.read_csv('./data/valid_newfeature.csv')

In [2]:
Samp_per=0.1

In [3]:
# #Samp_per should be in 0.001,0.01,0.025,0.05,0.075,0.1
# Samp_per=0.025
# # bids which click =1
# df1 = Train_df[Train_df['click']==1]
# # bids which click =0
# df0 = Train_df[Train_df['click']==0]
# # get 10% of click =0 data, down sampling
# df2=df0.sample(frac=Samp_per)
# # combine 10% of click =0 data and all click=1, down sampling
# combined_df = pd.concat([df1, df2],axis=0)
# combined_df.head()
# combined_df.shape
# #new trainning set shape is(62523, 127)

In [4]:
# combined_df.to_csv('/Users/weisihan/Desktop/ourtrainingset_Simple_XGBoost_model_25.csv', encoding='utf-8', index=False)

In [4]:
combined_df = pd.read_csv('./train_sample0.1_3.csv')

In [5]:
cols=list(Valid_df.columns)[1:]

In [6]:
train_y = combined_df['click']
train_x = combined_df[cols]

In [7]:
import xgboost

xgb = xgboost.XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=6,
 min_child_weight=7,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.01,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
#modelfit(xgb4, train, predictors)

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
#train xgboost model
import xgboost
X = train_x
y = train_y
# train the default model
#model = xgboost.XGBClassifier()
model = xgb
model.fit(X, y)
#14:15-14:16

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=7, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0.01, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [9]:
from sklearn.externals import joblib

filename = 'XGBoost-0.1-170.sav'
joblib.dump(model, filename)

['XGBoost-0.1-170.sav']

In [11]:
# from sklearn.externals import joblib
# model = joblib.load('train_sample01_3_xg.sav')

In [12]:
#test the default model
valid_x = Valid_df[cols]
valid_predictions = model.predict_proba(valid_x)
valid_predictions_class = model.predict(valid_x)

In [13]:
valid_predictions[:,1]

array([ 0.0018768 ,  0.00114815,  0.00123593, ...,  0.01032168,
        0.00376167,  0.00259068], dtype=float32)

In [14]:
valid_result = pd.DataFrame(columns=['click', 'click_pred_per', 'click_pred_class'])  
valid_result['click']= Valid_df['click'].copy()
valid_result['click_pred_per'] = valid_predictions[:,1]
valid_result['click_pred_class'] = valid_predictions_class
valid_result.head()

click  click_pred_per  click_pred_class
0      0        0.001877                 0
1      0        0.001148                 0
2      0        0.001236                 0
3      0        0.001847                 0
4      0        0.001121                 0

## calibrate pCTR

In [15]:
def recalibrate(p,w):
    q = p/(p+(1-p)/w)
    return q

In [16]:
valid_result['click_pred_per']=recalibrate(valid_result['click_pred_per'],Samp_per)
valid_result.head()

click  click_pred_per  click_pred_class
0      0        0.000188                 0
1      0        0.000115                 0
2      0        0.000124                 0
3      0        0.000185                 0
4      0        0.000112                 0

 随便看一下

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


y_test = valid_result['click']
y_pred_class = valid_result['click_pred_class']

print(confusion_matrix(y_test, y_pred_class))

confusion = confusion_matrix(y_test, y_pred_class)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP:", TP)
print ("TN:", TN)
print ("FP:", FP)
print ("FN:", FN)

#准确率、识别率（Classification Accuracy）：分类器正确分类的比例
#print ((TP+TN) / float(TP+TN+FN+FP))
print ("Accuracy: %.6f%%" % (accuracy_score(y_test, y_pred_class)* 100.0) )

#灵敏性（Sensitivity），也称为真正例识别率、召回率（Recall）：正确识别的正例数据在实际正例数据中的百分比
#print (TP / float(TP+FN))
#recall = metrics.recall_score(y_test, y_pred_class)
print ("Recall: %.6f%%" % (recall_score(y_test, y_pred_class)* 100.0))

#特效性（Specificity），也称为真负例率：正确识别的负例数据在实际负例数据中的百分比
print ("Specificity: %.6f%%" % (TN / float(TN+FP)* 100.0))

#假阳率（False Positive Rate）：实际值是负例数据，预测错误的百分比
#print (FP / float(TN+FP))
specificity = TN / float(TN+FP)
print ("False Positive Rate: %.6f%%" %((1 - specificity)*100.0))

#精度（Precision）：看做精确性的度量，即标记为正类的数据实际为正例的百分比
#print (TP / float(TP+FP))
precision = precision_score(y_test, y_pred_class)
print ("Precision: %.6f%%" %(precision*100.0))

#F1（平衡F分数）：精确率和召回率的调和平均数
#print (TP / float(TP+FP))
f1 = f1_score(y_test, y_pred_class)
print ("F1 score: %.6f%%" %(f1*100.0))



[[303600    123]
 [   138     64]]
TP: 64
TN: 303600
FP: 123
FN: 138
Accuracy: 99.914124%
Recall: 31.683168%
Specificity: 99.959503%
False Positive Rate: 0.040497%
Precision: 34.224599%
F1 score: 32.904884%


## Find Base Bid ----QR's Code

In [18]:
df_valid = pd.read_csv("./we_data/validation.csv")

In [19]:
# parameters
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df_qr = df_valid.copy()
test_predictions=valid_result['click_pred_per']
#test_predictions1=test_result['click_pred_per']

avgCTR = 1793/2430981 #train set
#tuning...

In [20]:
def generate_bidprice(base_bid, test_predictions):
    avgCTR = 1793/2430981#train set
    bidprices = base_bid*test_predictions/avgCTR
    return bidprices

In [21]:
def Prob3(Train_df,payprice,base_bid,test_predictions):
    Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
    
    #if count_break = 5 break
    count_break = 0
    
    current_budget = total_budget
    Train_df['win'] = 0 #initialize win label
    bidprices = generate_bidprice(base_bid, test_predictions)
    #start biding...
    for i in Train_df.index:
    #for i in range(Train_df.shape[0]):
#         if current_budget >= bidprices[i]:
#         if current_budget > 0:
            #generate bidprice:
        current_bidprice = bidprices[i]
        #consider budget
        if current_budget <= current_bidprice:              
            #count lack budget times
            count_break = count_break + 1
            print('lack of budget!',"current_budget:",current_budget,";current_bidprice:",current_bidprice,';count_break:',count_break)
            if count_break > 5:
                break  
            #reset bidprice
            current_bidprice = current_budget
        else:
            count_break = 0
            

        #Winning criterion:
        current_payprice = payprice[i]
        if current_bidprice > current_payprice:
            Train_df.at[i, 'win'] = int(1)
            current_budget = current_budget - current_payprice
        else:
            Train_df.at[i, 'win'] = int(0)

    #biding end        
    
    Train_df_new = Train_df[Train_df['win']!=0]
    #wintimes,_ = Train_df_new.shape
    #print('wintimes:',wintimes)
    #print('current_budget',current_budget)

    Clk = Train_df_new['click'].sum()
    Imp = Train_df_new['click'].count()
    CTR = Clk/Imp
    Spd = total_budget - current_budget
    CPM = 1000*Spd/Imp
    eCPC = Spd/Clk
    
    Train_Const_summ.loc[0,'Clicks']=Clk
    Train_Const_summ.loc[0,'Spend']=Spd
    Train_Const_summ.loc[0,'CTR']=CTR
    Train_Const_summ.loc[0,'CPM']=CPM
    Train_Const_summ.loc[0,'eCPC']=eCPC

    print(Train_Const_summ)
    
        
    return Train_df,current_budget

In [22]:
def calculate_click(base_bid,Train_df,payprice,test_predictions):
    print('base_bid:',base_bid)
    result_df,current_budget = Prob3(Train_df,payprice,base_bid,test_predictions)
    print('current_budget:',current_budget)
    Train_df_new = result_df[result_df['win']!=0]
    wintimes,_ = Train_df_new.shape
    print('wintimes:',wintimes)
    print('----------------')
    Clk = Train_df_new['click'].sum()    
    return Clk

In [23]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mpld3
import numpy as np

In [24]:
base_bids = np.arange(120,150,1)#区间可以适当左右调整一下，但大概就是这个范围
Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
fig = plt.figure()
print(max(Clks))
plt.plot(base_bids,Clks)
mpld3.display(fig)

base_bid: 120
       CPM         CTR Clicks    Spend   eCPC
0  44201.6  0.00131209    168  5659576  33688
current_budget: 590424
wintimes: 128040
----------------
base_bid: 121
       CPM         CTR Clicks    Spend     eCPC
0  44340.2  0.00130365    168  5714082  34012.4
current_budget: 535918
wintimes: 128869
----------------
base_bid: 122
       CPM         CTR Clicks    Spend     eCPC
0  44449.7  0.00129551    168  5764194  34310.7
current_budget: 485806
wintimes: 129679
----------------
base_bid: 123
       CPM         CTR Clicks    Spend     eCPC
0  44577.3  0.00128714    168  5818313  34632.8
current_budget: 431687
wintimes: 130522
----------------
base_bid: 124
       CPM         CTR Clicks    Spend     eCPC
0  44702.4  0.00127934    168  5870229  34941.8
current_budget: 379771
wintimes: 131318
----------------
base_bid: 125
       CPM         CTR Clicks    Spend     eCPC
0  44829.3  0.00127185    168  5921549  35247.3
current_budget: 328451
wintimes: 132091
----------------
ba

lack of budget! current_budget: 3 ;current_bidprice: 23.3450703254 ;count_break: 4
lack of budget! current_budget: 3 ;current_bidprice: 102.936880633 ;count_break: 5
lack of budget! current_budget: 3 ;current_bidprice: 16.3394320342 ;count_break: 6
       CPM         CTR Clicks    Spend     eCPC
0  46340.2  0.00113441    153  6249997  40849.7
current_budget: 3
wintimes: 134872
----------------
base_bid: 139
lack of budget! current_budget: 44248 ;current_bidprice: 48401.5638393 ;count_break: 1
lack of budget! current_budget: 2154 ;current_bidprice: 2385.05070787 ;count_break: 1
lack of budget! current_budget: 385 ;current_bidprice: 718.80223559 ;count_break: 1
lack of budget! current_budget: 249 ;current_bidprice: 266.702652576 ;count_break: 1
lack of budget! current_budget: 88 ;current_bidprice: 216.420945264 ;count_break: 1
lack of budget! current_budget: 57 ;current_bidprice: 62.3086153583 ;count_break: 1
lack of budget! current_budget: 57 ;current_bidprice: 58.1201691617 ;count_brea

lack of budget! current_budget: 11 ;current_bidprice: 29.0984561644 ;count_break: 4
lack of budget! current_budget: 11 ;current_bidprice: 142.521745663 ;count_break: 5
lack of budget! current_budget: 11 ;current_bidprice: 16.071617281 ;count_break: 6
     CPM         CTR Clicks    Spend     eCPC
0  47226  0.00112587    149  6249989  41946.2
current_budget: 11
wintimes: 132342
----------------
base_bid: 147
lack of budget! current_budget: 40731 ;current_bidprice: 50814.9322262 ;count_break: 1
lack of budget! current_budget: 30235 ;current_bidprice: 83267.9746618 ;count_break: 1
lack of budget! current_budget: 1867 ;current_bidprice: 2331.13887647 ;count_break: 1
lack of budget! current_budget: 221 ;current_bidprice: 802.598928711 ;count_break: 1
lack of budget! current_budget: 58 ;current_bidprice: 78.1838040155 ;count_break: 1
lack of budget! current_budget: 52 ;current_bidprice: 12181.5065078 ;count_break: 1
lack of budget! current_budget: 28 ;current_bidprice: 51.2649806723 ;count_br

In [25]:
base_bids = np.arange(128,133,1)#区间可以适当左右调整一下，但大概就是这个范围
Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
fig = plt.figure()
print(max(Clks))
plt.plot(base_bids,Clks)
mpld3.display(fig)

base_bid: 128
       CPM         CTR Clicks    Spend     eCPC
0  45186.8  0.00125737    169  6073418  35937.4
current_budget: 176582
wintimes: 134407
----------------
base_bid: 129
       CPM         CTR Clicks    Spend     eCPC
0  45312.9  0.00125788    170  6123945  36023.2
current_budget: 126055
wintimes: 135148
----------------
base_bid: 130
       CPM         CTR Clicks    Spend     eCPC
0  45431.7  0.00125108    170  6173393  36314.1
current_budget: 76607
wintimes: 135883
----------------
base_bid: 131
       CPM         CTR Clicks    Spend     eCPC
0  45557.4  0.00124407    170  6225329  36619.6
current_budget: 24671
wintimes: 136648
----------------
base_bid: 132
lack of budget! current_budget: 48430 ;current_bidprice: 69274.1007358 ;count_break: 1
lack of budget! current_budget: 37 ;current_bidprice: 37.8710918959 ;count_break: 1
lack of budget! current_budget: 37 ;current_bidprice: 67.1373013385 ;count_break: 1
lack of budget! current_budget: 37 ;current_bidprice: 130.0325904

In [26]:
basebid = 130

## Test + Submission

In [27]:
Test_df = pd.read_csv('./data/mytestset.csv')
Group=  pd.read_csv('./we_data/Group_xx.csv')

In [28]:
Test_df = Test_df[['slotwidth', 'slotheight', 'slotprice', 'android', 'ios',
       'linux', 'mac', 'other', 'windows', 'chrome', 'firefox', 'ie',
       'maxthon', 'opera', 'other.1', 'safari', 'sogou', 'theworld',
       '10006', '10024', '10031', '10048', '10052', '10057', '10059',
       '10063', '10067', '10074', '10075', '10076', '10077', '10079',
       '10083', '10093', '10102', '10110', '10111', '10114', '10115',
       '10116', '10117', '10118', '10120', '10123', '10125', '10126',
       '10127', '10129', '10130', '10131', '10133', '10138', '10140',
       '10142', '10145', '10146', '10147', '10148', '10149', '10684',
       '11092', '11278', '11379', '11423', '11512', '11576', '11632',
       '11680', '11724', '11944', '13042', '13403', '13496', '13678',
       '13776', '13800', '13866', '13874', '14273', '15398', '16593',
       '16617', '16661', '16706', '16751', '16753', 'hour_sin', 'hour_cos',
       'city_per', 'region_per', 'weekday_0', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'adexchange_1.0', 'adexchange_2.0', 'adexchange_3.0',
       'adexchange_4.0', 'adexchange_nan', 'slotvisibility_0',
       'slotvisibility_1', 'slotvisibility_2', 'slotvisibility_255',
       'slotvisibility_FifthView', 'slotvisibility_FirstView',
       'slotvisibility_FourthView', 'slotvisibility_Na',
       'slotvisibility_OtherView', 'slotvisibility_SecondView',
       'slotvisibility_ThirdView', 'slotformat_0', 'slotformat_1',
       'slotformat_5', 'slotformat_Na', 'advertiser_1458',
       'advertiser_2259', 'advertiser_2261', 'advertiser_2821',
       'advertiser_2997', 'advertiser_3358', 'advertiser_3386',
       'advertiser_3427', 'advertiser_3476']]

In [29]:
test_x = Test_df
test_predictions = model.predict_proba(test_x)
test_predictions_class = model.predict(test_x)

In [30]:
test_result = pd.DataFrame(columns=['bidprice', 'click_pred_per', 'click_pred_class'])  
test_result['click_pred_per'] = test_predictions[:,1]
test_result['click_pred_class'] = test_predictions_class
test_result.head()

bidprice  click_pred_per  click_pred_class
0      NaN        0.006802                 0
1      NaN        0.014516                 0
2      NaN        0.001122                 0
3      NaN        0.005100                 0
4      NaN        0.005430                 0

In [31]:
test_result['click_pred_per']=recalibrate(test_result['click_pred_per'],Samp_per)
test_result.head()

bidprice  click_pred_per  click_pred_class
0      NaN        0.000684                 0
1      NaN        0.001471                 0
2      NaN        0.000112                 0
3      NaN        0.000512                 0
4      NaN        0.000546                 0

In [32]:
base_bid=130
#这个变成你测出来最高clk的basebid！！！！！！！
avgCTR = 1793/2430981#train set
test_result['bidprice'] = base_bid*test_result['click_pred_per']/avgCTR
test_result.head()

bidprice  click_pred_per  click_pred_class
0  120.619973        0.000684                 0
1  259.239518        0.001471                 0
2   19.790194        0.000112                 0
3   90.304434        0.000512                 0
4   96.169556        0.000546                 0

In [33]:
Group['bidprice']=test_result['bidprice']
Group.head()

bidid    bidprice
0  366c563de7d90feb9d4dab53e795a93fb3157387  120.619973
1  29167d4caa719788b5a342dbaa25151d53121f80  259.239518
2  ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e   19.790194
3  844c2da00d45315f20b748ec131c26ee99a7cbc7   90.304434
4  c6017f0ad0c44d7d0c9b62583ea863f28941c0ca   96.169556

In [34]:
Group.to_csv('XGBoost-0.1-170.csv', encoding='utf-8', index=False)